# Toric geometry in `Oscar.jl`

This notebook gives an overview of functionality in `Oscar.jl` that is relevant to toric geometry. It is divided into two parts, one on polyhedral geometry and one on toric geometry.

We will mainly deal with *normal* toric varieties, since these correspond to combinatorially "easy" objects.

In [3]:
using Oscar

## Polyhedral geometry

The main objects one needs on the polyhedral side are cones, fans, and polyhedra. We will give a short overview on how to construct these and ask for various properties.

### Cones

We will make the cone associated to the two-dimensional cyclic quotient singularity $Y_{2,5}$. You can just take the following as a definition.

In [6]:
Y_2_5 = positive_hull([-2 5; 1 0])
rays(Y_2_5)

2-element VectorIterator{RayVector{Polymake.Rational}}:
 [-1, 5/2]
 [1, 0]

In [7]:
facets(Y_2_5)

2-element HalfspaceIterator{Halfspace}:
 The Halfspace of R^2 described by
1: -5*x₁ - 2*x₂ ≦ 0

 The Halfspace of R^2 described by
1: -x₂ ≦ 0


To dualize the cone, one needs to access `polymake` functionality directly. This will be wrapped in the future, however it is useful to demonstrate how this can be done for any other functionality that you might encounter which is not attached yet.

In [11]:
Y_2_5_dual = Cone(Polymake.polytope.polarize(Y_2_5.pm_cone))

A polyhedral cone in ambient dimension 2

In [12]:
rays(Y_2_5_dual)

2-element VectorIterator{RayVector{Polymake.Rational}}:
 [1, 2/5]
 [0, 1]

The Hilbert basis of the dual cone gives rise to the toric ideal defining `Y_{2,5}`.

In [13]:
hilbert_basis(Y_2_5_dual)

4-element VectorIterator{PointVector{Polymake.Integer}}:
 [5, 2]
 [0, 1]
 [2, 1]
 [1, 1]

Again to get at the toric ideal one currently has to go via `polymake`'s property system. This will also be wrapped in the future.

In [15]:
Y_2_5_dual.pm_cone.CONE_TORIC_IDEAL.BINOMIAL_GENERATORS

pm::Matrix<long>
-1 -1 2 1
-1 0 3 -1
0 -1 -1 2


### Polyhedra

We start by constructing a polyhedron as the convex hull of some points and then ask for some properties of it.

In [16]:
hexagon = convex_hull([0 0; 1 0; 2 1; 2 2; 1 2; 0 1])
vertices(hexagon)

6-element VectorIterator{PointVector{Polymake.Rational}}:
 [0, 0]
 [1, 0]
 [2, 1]
 [2, 2]
 [1, 2]
 [0, 1]

In [19]:
isnormal(hexagon)

true

In [20]:
lattice_points(hexagon)

7-element VectorIterator{PointVector{Polymake.Integer}}:
 [0, 0]
 [2, 1]
 [1, 2]
 [1, 0]
 [1, 1]
 [0, 1]
 [2, 2]

### Polyhedral fans

We continue with the hexagon and construct its normal fan.

In [21]:
nfh = normal_fan(hexagon)

A polyhedral fan in ambient dimension 2

In [23]:
rays(nfh)

6-element VectorIterator{RayVector{Polymake.Rational}}:
 [1, 0]
 [0, 1]
 [1, -1]
 [-1, 1]
 [-1, 0]
 [0, -1]

In [24]:
maximal_cones(nfh)

Oscar.MaximalConeIterator(A polyhedral fan in ambient dimension 2)

In [26]:
for mc in maximal_cones(nfh)
    println(rays(mc))
end

RayVector{Polymake.Rational}[[1, 0], [0, 1]]
RayVector{Polymake.Rational}[[0, 1], [-1, 1]]
RayVector{Polymake.Rational}[[-1, 1], [-1, 0]]
RayVector{Polymake.Rational}[[-1, 0], [0, -1]]
RayVector{Polymake.Rational}[[1, -1], [0, -1]]
RayVector{Polymake.Rational}[[1, 0], [1, -1]]


# Toric geometry